### https://towardsdatascience.com/introduction-to-weight-quantization-2494701b9c0c

In [ ]:
git clone https://github.com/TimDettmers/bitsandbytes.git && cd bitsandbytes/
pip install -r requirements-dev.txt
cmake -DCOMPUTE_BACKEND=cuda -S .
make
pip install -e .   # `-e` for "editable" install, when developing BNB (otherwise leave that out)

In [ ]:
!pip install bitsandbytes

يمكنك تغيير هذه القيمة باستخدام المعلمة. على سبيل المثال ، إذا كنت ترغب في تحسين المعلمات إلى 8 بت فقط إذا كان الحد الأدنى للحجم هو 16384 قيمة (يوصى باستخدام مضاعفات 4096):min_8bit_size

In [ ]:
import bitsandbytes as bnb

adam = bnb.optim.Adam8bit(model.parameters(), min_8bit_size=16384)

In [ ]:
import bitsandbytes as bnb

adam = bnb.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.995), optim_bits=32, percentile_clipping=5)

In [ ]:
import torch
import bitsandbytes as bnb

mng = bnb.optim.GlobalOptimManager.get_instance()

model = MyModel()
mng.register_parameters(model.parameters())

In [ ]:
model = model.cuda()
# use 8-bit optimizer states for all parameters
adam = bnb.optim.Adam(model.parameters(), lr=0.001, optim_bits=8)

# override the parameter model.fc1.weight now uses 32-bit Adam
mng.override_config(model.fc1.weight, "optim_bits", 32)

In [ ]:
mng.override_config([model.special.weight, model.also_special.weight],
                    key_value_dict ={'is_sparse': True, 'lr': 1e-5, 'betas'=(0.9, 0.98)})

In [ ]:
import torch
import torch.nn as nn

import bitsandbytes as bnb
from bnb.nn import Linear8bitLt

fp16_model = nn.Sequential(
    nn.Linear(64, 64),
    nn.Linear(64, 64)
)

int8_model = nn.Sequential(
    Linear8bitLt(64, 64, has_fp16_weights=False),
    Linear8bitLt(64, 64, has_fp16_weights=False)
)

int8_model.load_state_dict(fp16_model.state_dict())
int8_model = int8_model.to(0) # Quantization happens here

In [ ]:
import torch
import torch.nn as nn

import bitsandbytes as bnb
from bnb.nn import Linear4bit

fp16_model = nn.Sequential(
    nn.Linear(64, 64),
    nn.Linear(64, 64)
)

quantized_model = nn.Sequential(
    Linear4bit(64, 64),
    Linear4bit(64, 64)
)

quantized_model.load_state_dict(fp16_model.state_dict())
quantized_model = quantized_model.to(0) # Quantization happens here

In [ ]:
quant_type = 'fp4'
FP4 وNF4.

class bitsandbytes.nn.Params4bit
<
source
>
( data: Optional = Nonerequires_grad = Falsequant_state: Optional = Noneblocksize: int = 64compress_statistics: bool = Truequant_type: str = 'fp4'quant_storage: dtype = torch.uint8module: Optional = Nonebnb_quantized: bool =




  ( paramslrmomentum = 0dampening = 0weight_decay = 0nesterov = Falseoptim_bits = 32args = Nonemin_8bit_size = 4096percentile_clipping = 100block_wise = True )

__init__(
    self,
    in_features: int,
    out_features: int,

In [ ]:
import torch
import torch.nn as nn

import bitsandbytes as bnb
from bnb.nn import Linear4bit



fp16_model = nn.Sequential(
    nn.Linear(64, 64),
    nn.Linear(64, 64)
)


quantized_model = nn.Sequential(
    Linear4bit(64, 64),
    Linear4bit(64, 64)
)

quantized_model.load_state_dict(fp16_model.state_dict())
quantized_model = quantized_model.to(0) # Quantization happens here

https://huggingface.co/blog/hf-bitsandbytes-integration

https://huggingface.co/docs/bitsandbytes/reference/nn/linear4bit

https://huggingface.co/docs/bitsandbytes/explanations/resources

In [1]:
!pip install -q bitsandbytes>=0.39.0
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.5 MB/s eta 0:00:00


شغال

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)

# Set device to CPU for now
device = 'cpu'

# Load model and tokenizer
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Print model size
print(f"Model size: {model.get_memory_footprint():,} bytes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

شغل

In [6]:
weights = model.transformer.h[0].attn.c_attn.weight.data
print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

Original weights:
tensor([[-0.4738, -0.2614, -0.0978,  ...,  0.0513, -0.0584,  0.0250],
        [ 0.0874,  0.1473,  0.2387,  ..., -0.0525, -0.0113, -0.0156],
        [ 0.0039,  0.0695,  0.3668,  ...,  0.1143,  0.0363, -0.0318],
        ...,
        [-0.2592, -0.0164,  0.1991,  ...,  0.0095, -0.0516,  0.0319],
        [ 0.1517,  0.2170,  0.1043,  ...,  0.0293, -0.0429, -0.0475],
        [-0.4100, -0.1924, -0.2400,  ..., -0.0046,  0.0070,  0.0198]])


NameError: name 'absmax_quantize' is not defined

شغال

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define absmax_quantize function
def absmax_quantize(x, bit_width=8):
  """Quantizes a tensor using absmax quantization.

  Args:
    x: The input tensor.
    bit_width: The number of bits to use for quantization.

  Returns:
    A tuple containing the quantized tensor and the quantization scale.
  """
  # Calculate the maximum absolute value
  absmax = torch.max(torch.abs(x))

  # Calculate the quantization scale
  scale = (2**(bit_width - 1) - 1) / absmax

  # Quantize the tensor
  quantized_x = torch.round(x * scale)

  return quantized_x, scale

# Define zeropoint_quantize function
def zeropoint_quantize(x, bit_width=8):
  """Quantizes a tensor using zero-point quantization.

  Args:
    x: The input tensor.
    bit_width: The number of bits to use for quantization.

  Returns:
    A tuple containing the quantized tensor and the quantization scale.
  """
  # Calculate the minimum and maximum values
  min_val = torch.min(x)
  max_val = torch.max(x)

  # Calculate the quantization scale
  scale = (max_val - min_val) / (2**bit_width - 1)

  # Calculate the zero point
  zero_point = torch.round(-min_val / scale)

  # Quantize the tensor
  quantized_x = torch.round(x / scale + zero_point)

  return quantized_x, scale

# Extract weights of the first layer
weights = model.transformer.h[0].attn.c_attn.weight.data
print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

Original weights:
tensor([[-0.4738, -0.2614, -0.0978,  ...,  0.0513, -0.0584,  0.0250],
        [ 0.0874,  0.1473,  0.2387,  ..., -0.0525, -0.0113, -0.0156],
        [ 0.0039,  0.0695,  0.3668,  ...,  0.1143,  0.0363, -0.0318],
        ...,
        [-0.2592, -0.0164,  0.1991,  ...,  0.0095, -0.0516,  0.0319],
        [ 0.1517,  0.2170,  0.1043,  ...,  0.0293, -0.0429, -0.0475],
        [-0.4100, -0.1924, -0.2400,  ..., -0.0046,  0.0070,  0.0198]])

Absmax quantized weights:
tensor([[-21., -12.,  -4.,  ...,   2.,  -3.,   1.],
        [  4.,   7.,  11.,  ...,  -2.,  -1.,  -1.],
        [  0.,   3.,  16.,  ...,   5.,   2.,  -1.],
        ...,
        [-12.,  -1.,   9.,  ...,   0.,  -2.,   1.],
        [  7.,  10.,   5.,  ...,   1.,  -2.,  -2.],
        [-18.,  -9., -11.,  ...,  -0.,   0.,   1.]])

Zero-point quantized weights:
tensor([[108., 117., 125.,  ..., 131., 126., 130.],
        [133., 136., 140.,  ..., 127., 128., 128.],
        [129., 132., 146.,  ..., 134., 131., 128.],
        

https://towardsdatascience.com/introduction-to-weight-quantization-2494701b9c0c

شغال

In [8]:
import numpy as np
from copy import deepcopy

# Store original weights
weights = [param.data.clone() for param in model.parameters()]

# Create model to quantize
model_abs = deepcopy(model)

# Quantize all model weights
weights_abs = []
for param in model_abs.parameters():
    _, dequantized = absmax_quantize(param.data)
    param.data = dequantized
    weights_abs.append(dequantized)

# Create model to quantize
model_zp = deepcopy(model)

# Quantize all model weights
weights_zp = []
for param in model_zp.parameters():
    _, dequantized = zeropoint_quantize(param.data)
    param.data = dequantized
    weights_zp.append(dequantized)

In [9]:
import numpy as np
from copy import deepcopy

# Store original weights
weights = [param.data.clone() for param in model.parameters()]

# Create model to quantize
model_abs = deepcopy(model)

# Quantize only the weight tensors of specific layers
for name, param in model_abs.named_parameters():
    if "weight" in name and any(layer in name for layer in ["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"]): # Quantize weights of attention and MLP layers
        quantized, _ = absmax_quantize(param.data)
        param.data = quantized

# Create model to quantize
model_zp = deepcopy(model)

# Quantize only the weight tensors of specific layers
for name, param in model_zp.named_parameters():
    if "weight" in name and any(layer in name for layer in ["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"]): # Quantize weights of attention and MLP layers
        quantized, _ = zeropoint_quantize(param.data)
        param.data = quantized

شغال

In [10]:
def generate_text(model, input_text, max_length=50):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output = model.generate(inputs=input_ids,
                            max_length=max_length,
                            do_sample=True,
                            top_k=30,
                            pad_token_id=tokenizer.eos_token_id,
                            attention_mask=input_ids.new_ones(input_ids.shape))
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Generate text with original and quantized models
original_text = generate_text(model, "I have a dream")
absmax_text   = generate_text(model_abs, "I have a dream")
zp_text       = generate_text(model_zp, "I have a dream")

print(f"Original model:\n{original_text}")
print("-" * 50)
print(f"Absmax model:\n{absmax_text}")
print("-" * 50)
print(f"Zeropoint model:\n{zp_text}")

Original model:
I have a dream," he said. "You have a dream. The dream of a great guy. You dream that you'll be one day an NFL superstar. You dream that you'll be one day a quarterback in the NFL. You dream that
--------------------------------------------------
Absmax model:
I have a dream,*** in |,,,,,,,,:—, and..,,. Stro).–. (-))").—H,—.).–G..,.)Interested-).
--------------------------------------------------
Zeropoint model:
I have a dream the about at 3 the the in 4 at the a, to the on the over the of all to to, the O a one, about- the... ' M- M the, the 2, the at at 2 "


شغال

In [11]:
def calculate_perplexity(model, text):
    # Encode the text
    encodings = tokenizer(text, return_tensors='pt').to(device)

    # Define input_ids and target_ids
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

    # Loss calculation
    neg_log_likelihood = outputs.loss

    # Perplexity calculation
    ppl = torch.exp(neg_log_likelihood)

    return ppl

ppl     = calculate_perplexity(model, original_text)
ppl_abs = calculate_perplexity(model_abs, absmax_text)
ppl_zp  = calculate_perplexity(model_zp, absmax_text)

print(f"Original perplexity:  {ppl.item():.2f}")
print(f"Absmax perplexity:    {ppl_abs.item():.2f}")
print(f"Zeropoint perplexity: {ppl_zp.item():.2f}")

Original perplexity:  7.59
Absmax perplexity:    5361.89
Zeropoint perplexity: 8183.22


In [30]:
!pip install bitsandbytes

In [33]:
!pip install transformers accelerate bitsandbytes

شغال

In [12]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# import bitsandbytes as bnb # This might be optional depending on usage

# ... (rest of your code) ...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_int8 = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             load_in_8bit=True,
                                             )
print(f"Model size: {model_int8.get_memory_footprint():,} bytes")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Model size: 176,527,896 bytes


شغال

In [13]:
text_int8 = generate_text(model_int8, "I have a dream")

print(f"Original model:\n{original_text}")
print("-" * 50)
print(f"LLM.int8() model:\n{text_int8}")

Original model:
I have a dream," he said. "You have a dream. The dream of a great guy. You dream that you'll be one day an NFL superstar. You dream that you'll be one day a quarterback in the NFL. You dream that
--------------------------------------------------
LLM.int8() model:
I have a dream. I don't know what will come of it, but I am going to have to look for something that will be right. I haven't thought about it for a long time, but I have to try to get that thing


In [14]:
print(f"Perplexity (original):   {ppl.item():.2f}")

ppl = calculate_perplexity(model_int8, text_int8)
print(f"Perplexity (LLM.int8()): {ppl.item():.2f}")

Perplexity (original):   7.59
Perplexity (LLM.int8()): 7.87


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Check if CUDA is available
if device.type == 'cuda':
    print("CUDA is available. Using GPU for model loading.")
else:
    print("CUDA is not available. Using CPU. 8-bit optimization will not be available.")

model_int8 = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             load_in_8bit=True,
                                             )
print(f"Model size: {model_int8.get_memory_footprint():,} bytes")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


CUDA is not available. Using CPU. 8-bit optimization will not be available.


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# Generate text with quantized model
text_int8 = generate_text(model_int8, "I have a dream")

print(f"Original model:\n{original_text}")
print("-" * 50)
print(f"LLM.int8() model:\n{text_int8}")

In [ ]:
!pip install transformers accelerate bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# import bitsandbytes as bnb # This might be optional depending on usage

# ... (rest of your code) ...

In [ ]:
device_map = 'auto'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)

# Set device to CPU for now
#device = 'auto'
device_map = 'auto'

# Load model and tokenizer
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Print model size
print(f"Model size: {model.get_memory_footprint():,} bytes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

شغال aclrate

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
import torch

# إعداد Accelerator الذي يدير الأجهزة المتاحة (CPU و GPU)
accelerator = Accelerator()

# تحميل النموذج والمحول باستخدام device_map='auto' للتوزيع التلقائي على الأجهزة المتاحة
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)

# نقل النموذج والمحول إلى الجهاز المدعوم من Accelerator
model, tokenizer = accelerator.prepare(model, tokenizer)

# طباعة حجم النموذج بعد النقل
print(f"Model size: {model.get_memory_footprint():,} bytes")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

Model size: 32,112,140,544 bytes


In [2]:
weights = model.transformer.h[0].attn.c_attn.weight.data
print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

AttributeError: 'CohereForCausalLM' object has no attribute 'transformer'

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Assuming you are targeting the first layer's attention weights
weights = model.model.layers[0].self_attn.c_attn.weight.data
print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

AttributeError: 'CohereSdpaAttention' object has no attribute 'c_attn'

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Assuming you are targeting the first layer's attention weights
# Accessing SDPA attention weights for CohereForCausalLM
#weights = model.model.layers[0].self_attn.W_q.weight.data
# W_q, W_k, W_v are the query, key, and value weight matrices in SDPA
# You can select the desired weight matrix (W_q, W_k, or W_v)

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

Original weights:


NameError: name 'weights' is not defined

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Assuming you are targeting the first layer's attention weights
# Accessing SDPA attention weights for CohereForCausalLM
weights = model.model.layers[0].self_attn.W_q.weight.data  # Uncomment and assign weights
# W_q, W_k, W_v are the query, key, and value weight matrices in SDPA
# You can select the desired weight matrix (W_q, W_k, or W_v)

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Assuming you are targeting the first layer's attention weights
# Accessing SDPA attention weights for CohereForCausalLM
weights = model.model.layers[0].self_attn.W_q.weight.data  # Uncomment and assign weights
# W_q, W_k, W_v are the query, key, and value weight matrices in SDPA
# You can select the desired weight matrix (W_q, W_k, or W_v)

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

AttributeError: 'CohereSdpaAttention' object has no attribute 'W_q'

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Accessing SDPA attention weights for CohereForCausalLM
# Instead of W_q, check for attributes like W_out or a similar name
# This is because CohereSdpaAttention may have a different internal structure
# You may need to inspect the model architecture to find the correct attribute name
# Try printing model.model.layers[0].self_attn to see its attributes
try:
    weights = model.model.layers[0].self_attn.W_out.weight.data
except AttributeError:
    print("W_out attribute not found. Inspecting model.model.layers[0].self_attn:")
    print(model.model.layers[0].self_attn)
    # Based on the printed output, replace 'your_attribute_name' with the actual attribute
    # holding the desired weights
    weights = model.model.layers[0].self_attn.your_attribute_name.weight.data

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

W_out attribute not found. Inspecting model.model.layers[0].self_attn:
CohereSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): CohereRotaryEmbedding()
)


AttributeError: 'CohereSdpaAttention' object has no attribute 'your_attribute_name'

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Accessing SDPA attention weights for CohereForCausalLM
# Instead of W_q or W_out, we need to inspect the model's structure
# Print the self_attn layer to see its attributes
print(model.model.layers[0].self_attn)

# Based on the printed output, find the correct attribute name for weights
# For example, it might be 'W_qkv' or something similar
# Replace 'correct_attribute_name' with the actual attribute name you found
weights = model.model.layers[0].self_attn.correct_attribute_name.weight.data

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

CohereSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): CohereRotaryEmbedding()
)


AttributeError: 'CohereSdpaAttention' object has no attribute 'correct_attribute_name'

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Accessing SDPA attention weights for CohereForCausalLM
# Instead of W_q or W_out, we need to inspect the model's structure
# Print the self_attn layer to see its attributes
print(model.model.layers[0].self_attn)

# Based on the printed output, find the correct attribute name for weights
# For example, it might be 'W_qkv' or something similar
# Replace 'correct_attribute_name' with the actual attribute name you found
# Assuming 'W_qkv' is the correct attribute name based on inspection
weights = model.model.layers[0].self_attn.W_qkv.weight.data  # Replace 'W_qkv' if needed

print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

CohereSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): CohereRotaryEmbedding()
)


AttributeError: 'CohereSdpaAttention' object has no attribute 'W_qkv'

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, transformer
import torch

# Define absmax_quantize function
def absmax_quantize(x, bit_width=8):
  """Quantizes a tensor using absmax quantization.

  Args:
    x: The input tensor.
    bit_width: The number of bits to use for quantization.

  Returns:
    A tuple containing the quantized tensor and the quantization scale.
  """
  # Calculate the maximum absolute value
  absmax = torch.max(torch.abs(x))

  # Calculate the quantization scale
  scale = (2**(bit_width - 1) - 1) / absmax

  # Quantize the tensor
  quantized_x = torch.round(x * scale)

  return quantized_x, scale

# Define zeropoint_quantize function
def zeropoint_quantize(x, bit_width=8):
  """Quantizes a tensor using zero-point quantization.

  Args:
    x: The input tensor.
    bit_width: The number of bits to use for quantization.

  Returns:
    A tuple containing the quantized tensor and the quantization scale.
  """
  # Calculate the minimum and maximum values
  min_val = torch.min(x)
  max_val = torch.max(x)

  # Calculate the quantization scale
  scale = (max_val - min_val) / (2**bit_width - 1)

  # Calculate the zero point
  zero_point = torch.round(-min_val / scale)

  # Quantize the tensor
  quantized_x = torch.round(x / scale + zero_point)

  return quantized_x, scale

# Extract weights of the first layer
weights = model.transformer.h[0].attn.c_attn.weight.data
print("Original weights:")
print(weights)

# Quantize layer using absmax quantization
weights_abs_quant, _ = absmax_quantize(weights)
print("\nAbsmax quantized weights:")
print(weights_abs_quant)

# Quantize layer using absmax quantization
weights_zp_quant, _ = zeropoint_quantize(weights)
print("\nZero-point quantized weights:")
print(weights_zp_quant)

ImportError: cannot import name 'transformer' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [ ]:
import numpy as np
from copy import deepcopy

# Store original weights
weights = [param.data.clone() for param in model.parameters()]

# Create model to quantize
model_abs = deepcopy(model)

# Quantize all model weights
weights_abs = []
for param in model_abs.parameters():
    _, dequantized = absmax_quantize(param.data)
    param.data = dequantized
    weights_abs.append(dequantized)

# Create model to quantize
model_zp = deepcopy(model)

# Quantize all model weights
weights_zp = []
for param in model_zp.parameters():
    _, dequantized = zeropoint_quantize(param.data)
    param.data = dequantized
    weights_zp.append(dequantized)

In [ ]:
import numpy as np
from copy import deepcopy

# Store original weights
weights = [param.data.clone() for param in model.parameters()]

# Create model to quantize
model_abs = deepcopy(model)

# Quantize only the weight tensors of specific layers
for name, param in model_abs.named_parameters():
    if "weight" in name and any(layer in name for layer in ["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"]): # Quantize weights of attention and MLP layers
        quantized, _ = absmax_quantize(param.data)
        param.data = quantized

# Create model to quantize
model_zp = deepcopy(model)

# Quantize only the weight tensors of specific layers
for name, param in model_zp.named_parameters():
    if "weight" in name and any(layer in name for layer in ["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"]): # Quantize weights of attention and MLP layers
        quantized, _ = zeropoint_quantize(param.data)
        param.data = quantized

In [ ]:
def generate_text(model, input_text, max_length=50):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output = model.generate(inputs=input_ids,
                            max_length=max_length,
                            do_sample=True,
                            top_k=30,
                            pad_token_id=tokenizer.eos_token_id,
                            attention_mask=input_ids.new_ones(input_ids.shape))
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Generate text with original and quantized models
original_text = generate_text(model, "I have a dream")
absmax_text   = generate_text(model_abs, "I have a dream")
zp_text       = generate_text(model_zp, "I have a dream")

print(f"Original model:\n{original_text}")
print("-" * 50)
print(f"Absmax model:\n{absmax_text}")
print("-" * 50)
print(f"Zeropoint model:\n{zp_text}")

Original model:
I have a dream," he said. "You have a dream. The dream of a great guy. You dream that you'll be one day an NFL superstar. You dream that you'll be one day a quarterback in the NFL. You dream that
--------------------------------------------------
Absmax model:
I have a dream,*** in |,,,,,,,,:—, and..,,. Stro).–. (-))").—H,—.).–G..,.)Interested-).
--------------------------------------------------
Zeropoint model:
I have a dream the about at 3 the the in 4 at the a, to the on the over the of all to to, the O a one, about- the... ' M- M the, the 2, the at at 2 "


In [ ]:
def calculate_perplexity(model, text):
    # Encode the text
    encodings = tokenizer(text, return_tensors='pt').to(device)

    # Define input_ids and target_ids
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

    # Loss calculation
    neg_log_likelihood = outputs.loss

    # Perplexity calculation
    ppl = torch.exp(neg_log_likelihood)

    return ppl

ppl     = calculate_perplexity(model, original_text)
ppl_abs = calculate_perplexity(model_abs, absmax_text)
ppl_zp  = calculate_perplexity(model_zp, absmax_text)

print(f"Original perplexity:  {ppl.item():.2f}")
print(f"Absmax perplexity:    {ppl_abs.item():.2f}")
print(f"Zeropoint perplexity: {ppl_zp.item():.2f}")

Original perplexity:  7.59
Absmax perplexity:    5361.89
Zeropoint perplexity: 8183.22


In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# import bitsandbytes as bnb # This might be optional depending on usage

# ... (rest of your code) ...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_int8 = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             load_in_8bit=True,
                                             )
print(f"Model size: {model_int8.get_memory_footprint():,} bytes")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Model size: 176,527,896 bytes


In [ ]:
text_int8 = generate_text(model_int8, "I have a dream")

print(f"Original model:\n{original_text}")
print("-" * 50)
print(f"LLM.int8() model:\n{text_int8}")

Original model:
I have a dream," he said. "You have a dream. The dream of a great guy. You dream that you'll be one day an NFL superstar. You dream that you'll be one day a quarterback in the NFL. You dream that
--------------------------------------------------
LLM.int8() model:
I have a dream. I don't know what will come of it, but I am going to have to look for something that will be right. I haven't thought about it for a long time, but I have to try to get that thing


In [ ]:
print(f"Perplexity (original):   {ppl.item():.2f}")

ppl = calculate_perplexity(model_int8, text_int8)
print(f"Perplexity (LLM.int8()): {ppl.item():.2f}")

Perplexity (original):   7.59
Perplexity (LLM.int8()): 7.87


In [2]:
import gc
import torch

# إفراغ ذاكرة وحدة معالجة الرسومات
torch.cuda.empty_cache()

# إفراغ ذاكرة الوصول العشوائي
gc.collect()

0

شغال تسريع و8bit

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# إعداد Accelerator لإدارة الموارد (CPU و GPU)
accelerator = Accelerator()

# إعداد التحميل إلى 8 بت
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# تحميل النموذج والمحول باستخدام 8-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# نقل النموذج والمحول إلى الجهاز المدعوم من Accelerator
model, tokenizer = accelerator.prepare(model, tokenizer)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "How are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Response: How are you?
I am doing well, thanks for asking. I have been busy with work and taking care of my family. I have also been trying to get back into a regular workout routine. I have been going to the gym 


device_map={'': device}: يخصص جميع أجزاء النموذج للجهاز المحدد، سواء كان GPU أو CPU.
### تأكد من أن لديك ذاكرة كافية على GPU إذا كان النموذج بالكامل سيتم تشغيله هناك، أو أنه سيتم تقسيمه بمرونة بين الأجهزة عند استخدام device_map='auto'.

### شغال

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# إعداد التحميل إلى 8 بت مع تمكين التحميل التدريجي إلى CPU
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # تمكين التحميل التدريجي إلى CPU
)

# تحميل النموذج والمحول باستخدام 8-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map={'': device},  # تعيين جميع الأجزاء إلى GPU إن كانت متاحة
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "How are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Response: How are you?
I am doing well, thanks for asking. I have been busy with work and taking care of my family. I have also been trying to get back into a regular workout routine. I have been going to the gym 


### load_in_4bit

### شغال

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# إعداد التحميل إلى 4 بت مع تمكين التحميل التدريجي إلى CPU
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # تغيير إلى 4 بت
    llm_int8_enable_fp32_cpu_offload=True  # تمكين التحميل التدريجي إلى CPU
)

# تحميل النموذج والمحول باستخدام 4-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map={'': device},  # تعيين جميع الأجزاء إلى GPU إن كانت متاحة
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "How are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Response: How are you?
I am doing well, thanks for asking. I have been busy with work and taking care of my family. I have been trying to get back into the swing of things and get back into a routine. I have been


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_compute_dtype=torch.float16  # تغيير إلى float16
)


### شغال تسريع و 4بت

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# إعداد التحميل إلى 4 بت مع تمكين التحميل التدريجي إلى CPU
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # تغيير إلى 4 بت
    llm_int8_enable_fp32_cpu_offload=True,  # تمكين التحميل التدريجي إلى CPU
    bnb_4bit_compute_dtype=torch.float16  # تغيير إلى float16
)

# تحميل النموذج والمحول باستخدام 4-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map={'': device},  # تعيين جميع الأجزاء إلى GPU إن كانت متاحة
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "How are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Response: How are you?
I am doing well, thanks for asking. I have been busy with work and taking care of my family. I have been trying to get back into the swing of things and get back into a routine. I have been


### 4بت وتسريع

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# إعداد Accelerator لإدارة الموارد (CPU و GPU)
accelerator = Accelerator()

# إعداد التحميل إلى 4 بت
quantization_config = BitsAndBytesConfig(load_in_4bit=True)  # تغيير إلى 4 بت

# تحميل النموذج والمحول باستخدام 4-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# نقل النموذج والمحول إلى الجهاز المدعوم من Accelerator
model, tokenizer = accelerator.prepare(model, tokenizer)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "How are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Response: How are you?
I am doing well, thanks for asking. I have been busy with work and taking care of my family. I have been trying to get back into the swing of things and get back into a routine. I have been


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# إعداد Accelerator لإدارة الموارد (CPU و GPU)
accelerator = Accelerator()

# إعداد التحميل إلى 4 بت
quantization_config = BitsAndBytesConfig(load_in_4bit=True)  # تغيير إلى 4 بت

# تحميل النموذج والمحول باستخدام 4-bit quantization و device_map='auto'
model_id = 'CohereForAI/aya-expanse-8b'
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# نقل النموذج والمحول إلى الجهاز المدعوم من Accelerator
model, tokenizer = accelerator.prepare(model, tokenizer)

# إعداد الإدخال للمطالبة "How are you?"
input_text = "Who is Napoleon Bonaparte?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

# الحصول على استجابة النموذج
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# عرض الاستجابة
print(f"Response: {response}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Response: Who is Napoleon Bonaparte?
Napoleon Bonaparte was a French military leader and emperor who rose to prominence during the French Revolution and led France from 1799 to 1815. He is considered one of the greatest military commanders in history, and his legacy continues to shape European politics and culture.
Born on August 15, 1769, in Ajaccio, Corsica, Napoleon showed early signs of intelligence and ambition. He received a thorough education, excelling in mathematics and languages. After joining the French Army at 17, Napoleon quickly rose through the ranks due to his strategic brilliance and military prowess.
His rise to power began during the French Revolution, where he played a crucial role in suppressing royalist rebellions. In 1799, Napoleon staged a coup d'état, becoming the First Consul of France. He then embarked on a series of military campaigns, conquering much of Europe and establishing a vast empire.
As Emperor of the French, Napoleon implemented significant reforms